<a href="https://colab.research.google.com/github/KarlMh/projects/blob/main/UTT_dataset_use_case_LM_Vstudents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cas Leroy Merlin

L'objectif de ce cas est de construire un modèle capable de prédire si un utilisateur ayant visité le site web de Leroy Merlin va venir acheter en magasin.
On appelle cet effet, l'effet ROPO (Research Online, Purchase Offline)

Pour réaliser ce modèle nous allons suivre plusieurs étapes :

## Load data

In [ ]:
import pandas as pd
from google.colab import files
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
# Charge des fichiers locals dans le "repertoire" colab
uploaded = files.upload()

Saving utt_store_purchases_sample.csv to utt_store_purchases_sample.csv
Saving utt_web_visits_sample.csv to utt_web_visits_sample.csv
Saving sa-creds-c-utt-dataset.json to sa-creds-c-utt-dataset.json
Saving model.pickle to model.pickle


In [ ]:
# Lister les éléments du repertoire
!ls
# S'assurer que le repertoire contiennent les fichiers suivant:
# - utt_store_purchases_sample.csv
# - utt_web_visits_sample.csv
# - sa-creds-c-utt-dataset.json
# - model.pickle

model.pickle		     sample_data		     utt_web_visits_sample.csv
sa-creds-c-utt-dataset.json  utt_store_purchases_sample.csv


In [ ]:
# Stocker le nom des fichiers dans une variable pour le lire avec pandas
store_purchases_file = '/content/utt_store_purchases_sample.csv'
web_visits_file = '/content/utt_web_visits_sample.csv'

In [ ]:
# Lire le fichier csv des données store purchases dans une dataframe avec pandas
df_store_purchases = pd.read_csv(store_purchases_file, sep=',')
print("Dimension de la dataframe est : {}".format(df_store_purchases.shape)) # Donne la shape de la dataframe (nb lignes, nb colonnes)
df_store_purchases.head() # Afficher les n premières lignes de la dataframe

Dimension de la dataframe est : (85209, 8)


,user_id,date,family,class,type,product,product_id,price
0,49263,2020-02-26,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5
1,30940,2022-12-16,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5
2,11386,2020-02-18,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5
3,37749,2020-11-04,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5
4,14128,2021-02-10,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5


In [ ]:
# Lire le fichier csv des données web sessions dans une dataframe avec pandas
df_web_visits = pd.read_csv(web_visits_file, sep=',')
print("Dimension de la dataframe est : {}".format(df_web_visits.shape)) # Donne la shape de la dataframe (nb lignes, nb colonnes)
df_web_visits.head() # Afficher les n premières lignes de la dataframe

Dimension de la dataframe est : (83995, 9)


,family,class,type,web_visit_date,user_id,product,price,product_id,conversion
0,family_6,class_3,type_4,2020-03-27,1289.0,product_3,5,00043a7a-ba01-4653-a8c9-dfc1c256372d,0
1,family_6,class_3,type_4,2021-03-06,33463.0,product_3,5,00043a7a-ba01-4653-a8c9-dfc1c256372d,0
2,family_6,class_3,type_4,2021-12-07,2752.0,product_3,5,00043a7a-ba01-4653-a8c9-dfc1c256372d,0
3,family_6,class_3,type_4,2022-01-06,18620.0,product_3,5,00043a7a-ba01-4653-a8c9-dfc1c256372d,0
4,family_6,class_3,type_4,2020-11-03,82.0,product_3,5,00043a7a-ba01-4653-a8c9-dfc1c256372d,0


## Exploration données store purchases & web sites

### store purchase

In [ ]:
## Compter le nombre d'utilisateur unique dans la dataframe
print("-----STORE-PURCHASES-----")
nb_unique_user_id_purchase = df_store_purchases.nunique()
print(f"Nombre d'utilisateur unique (distinct) dans la dataframe : {nb_unique_user_id_purchase}")
print("-------------------------")

-----STORE-PURCHASES-----
Nombre d'utilisateur unique (distinct) dans la dataframe : user_id       34261
date           1126
family            7
class             9
type              4
product          37
product_id      308
price           148
dtype: int64
-------------------------


In [ ]:
## Compter le nombre de produit par utilisateur
products_per_user = df_store_purchases.groupby('user_id')['product_id'].nunique()
print("Nombre de produits par user_id :")
print(products_per_user.sort_values(ascending=False).head(10)) # Affiche les 10 user_id avec le plus grand nombre de product_id rattachés
print("-------------------------")

Nombre de produits par user_id :
user_id
8736     10
14756    10
15044    10
36462     9
23286     9
39205     9
2863      9
34691     9
16475     9
19889     9
Name: product_id, dtype: int64
-------------------------


In [ ]:
## Verifier s'il y a des valeurs nulles dans les colonnes de la dataframe et combien s'il y en existe
null_values = df_store_purchases.isnull().sum()
print("Nombre de valeurs nulles par colonne :")
print(null_values)
print("-------------------------")

Nombre de valeurs nulles par colonne :
user_id       0
date          0
family        0
class         0
type          0
product       0
product_id    0
price         0
dtype: int64
-------------------------


### Web visit

In [ ]:
print("-----WEB-VISITS-----")
## Compter le nombre d'utilisateur unique dans la dataframe
nb_unique_user_id_web = df_web_visits['user_id'].nunique()
print(f"Nombre d'utilisateur unique (distinct) dans la dataframe : {nb_unique_user_id_web}")
print("-------------------------")

-----WEB-VISITS-----
Nombre d'utilisateur unique (distinct) dans la dataframe : 7828
-------------------------


In [ ]:
## Afficher les différentes familles de produits vue
mod_family = df_web_visits.groupby('family')['product'].unique()
print("Les différentes familles de produits vue sur le site :")
print(mod_family)
print("-------------------------")


Les différentes familles de produits vue sur le site :
family
family_1    [product_13, product_9, product_2, product_8, ...
family_2    [product_11, product_18, product_20, product_1...
family_3    [product_1, product_18, product_11, product_5,...
family_4    [product_25, product_39, product_7, product_4,...
family_5    [product_3, product_13, product_2, product_19,...
family_6    [product_3, product_36, product_7, product_26,...
family_9    [product_14, product_5, product_18, product_22...
Name: product, dtype: object
-------------------------


In [ ]:
## Vérifier la plage de date pour les visites web
date_min = df_web_visits['web_visit_date'].min()
date_max = df_web_visits['web_visit_date'].max()
print(f"Plage de dates : {date_min} à {date_max}")

Plage de dates : 2020-01-03 à 2023-01-30


In [ ]:
## Si on souhaite pousser l'analyse, on peut vérifier s'il manque des dates entre la date min et la date max
### Créer une séquence complète de dates entre min et max
complete_date_range = pd.date_range(start= date_min, end= date_max, freq='D')
### Convertir la colonne en datetime
### Extraire les dates uniques présentes dans le DataFrame
existing_dates = df_web_visits['web_visit_date'].unique()
### Vérifier s'il manque des dates en comparant la séquence complète avec les dates existantes
missing_dates = set(complete_date_range) - set(existing_dates)
if missing_dates:
    print(f"Il manque {len(missing_dates)} dates :")
    print(sorted(missing_dates))  # Affiche les dates manquantes
else:
    print("Il n'y a aucune date manquante entre la date minimale et la date maximale.")


Il n'y a aucune date manquante entre la date minimale et la date maximale.


## Feature engineering

Nous avons 2 tables distinctes avec des id visiteur qui ont réalisé des visites en ligne et des id visiteur qui ont procédé à l'achat en magasin.
Notre objectif est de préparer un dataset unique avec les variables explicatives et la variable que l'on cherche à expliquer.

La première etape de la modélisation sera d'entrainer et de tester notre modèle sur une base connue. C'est à dire avec des visiteurs qui ont converti en magasin (procédé à l'achat en magasin)

### Méthode 1 à privilégier

In [ ]:
# Créer une liste de visiteur unique en magasin
# Filtrer les user_id non nuls dans store_purchases
distinct_users = df_store_purchases['user_id'].dropna().nunique()


In [ ]:
# Ne garder que les visiteurs qui ont réalisé un achat en magasin
df_type_visits = df_web_visits[df_web_visits['user_id'].isin(df_store_purchases['user_id'])].drop_duplicates(
    subset=['family', 'class', 'type', 'web_visit_date', 'user_id']
)

#### Créer une variable comptant le nombre de visite web réalisé dans les 30 jours

In [ ]:
# Joindre la table sur elle-même pour trouver les visites dans les 30 jours précédents
df_visit_count_30d = df_type_visits.merge (
    df_type_visits,
    on=['family', 'class', 'type', 'user_id'],
    suffixes=('', '_b')
)

In [ ]:
# Convertir les dates au format datetime
df_visit_count_30d['web_visit_date'] = pd.to_datetime(df_visit_count_30d['web_visit_date'])
df_visit_count_30d['web_visit_date_b'] = pd.to_datetime(df_visit_count_30d['web_visit_date_b'])

In [ ]:
# Filtre sur l'intervalle de dates pour ne garder que les visites web dans les 30 derniers jours
result = df_visit_count_30d[
    (df_visit_count_30d['web_visit_date_b'] >= (df_visit_count_30d['web_visit_date'] - pd.Timedelta(days=30)) ) &
    (df_visit_count_30d['web_visit_date_b'] <= df_visit_count_30d['web_visit_date'] )
]

In [ ]:
# Compte le nombre de visites dans les 30 derniers jours
df_visit_count_30d = result.groupby(['family', 'class', 'type', 'user_id', 'web_visit_date']).size().reset_index(name='visit_count_30d')
df_visit_count_30d


,family,class,type,user_id,web_visit_date,visit_count_30d
0,family_1,class_2,type_1,212.0,2021-08-29,1
1,family_1,class_2,type_1,334.0,2021-08-07,1
2,family_1,class_2,type_1,366.0,2021-01-27,1
3,family_1,class_2,type_1,429.0,2021-11-21,1
4,family_1,class_2,type_1,568.0,2022-08-19,1
...,...,...,...,...,...,...
15962,family_9,class_9,type_1,35636.0,2021-01-29,1
15963,family_9,class_9,type_1,35636.0,2021-09-13,1
15964,family_9,class_9,type_1,35636.0,2022-09-28,1
15965,family_9,class_9,type_1,38784.0,2020-06-21,1


In [ ]:
# Associer les informations de ventes en magasins aux informations des visites web
df_merged = df_visit_count_30d.merge(
    df_store_purchases,
    on='user_id',
    suffixes=('_web', '_store')
)

#### Créer notre variable que l'on cherche à prédire

In [ ]:
# Filtre les lignes selon si l'utilisateur a acheté en magasin ou non (si un achat a été fait 30j après la visite en magasin)
df_merged['store_conversion'] = (
    (df_merged['date'] >= df_merged['web_visit_date']) &
    (df_merged['date'] <= df_merged['web_visit_date'] + pd.Timedelta(days=30))
).astype(int)

df_merged.head()


,family_web,class_web,type_web,user_id,web_visit_date,visit_count_30d,date,family_store,class_store,type_store,product,product_id,price,store_conversion
0,family_1,class_2,type_1,212.0,2021-08-29,1,2021-06-24,family_9,class_8,type_2,product_9,26cd5030-e547-4725-8894-031f352330af,1,0
1,family_1,class_2,type_1,334.0,2021-08-07,1,2020-12-14,family_1,class_3,type_1,product_1,19f31143-41d6-4e4a-aedf-1eca0d014971,13,0
2,family_1,class_2,type_1,366.0,2021-01-27,1,2022-02-06,family_5,class_3,type_3,product_24,1875a249-e60b-4254-92c7-95289cf4f9e9,165,0
3,family_1,class_2,type_1,366.0,2021-01-27,1,2022-04-17,family_4,class_5,type_2,product_27,2488496e-d547-4d41-aa45-3a6b2f4c18e8,2,0
4,family_1,class_2,type_1,429.0,2021-11-21,1,2022-10-25,family_4,class_2,type_1,product_16,22a34829-f78b-4551-aae4-3466a6543981,87,0


In [ ]:
# Remplace NaN par 0 dans store_conversion (b.date null implique pas de conversion)
df_merged['store_conversion'] = df_merged['store_conversion'].fillna(0)


In [ ]:
# Supprimer les doublons et conserver les colonnes distinctes
df_result = df_merged.drop_duplicates(
    subset=['family_web', 'class_web', 'type_web', 'user_id', 'web_visit_date']
).copy()

df_result.head()

,family_web,class_web,type_web,user_id,web_visit_date,visit_count_30d,date,family_store,class_store,type_store,product,product_id,price,store_conversion
0,family_1,class_2,type_1,212.0,2021-08-29,1,2021-06-24,family_9,class_8,type_2,product_9,26cd5030-e547-4725-8894-031f352330af,1,0
1,family_1,class_2,type_1,334.0,2021-08-07,1,2020-12-14,family_1,class_3,type_1,product_1,19f31143-41d6-4e4a-aedf-1eca0d014971,13,0
2,family_1,class_2,type_1,366.0,2021-01-27,1,2022-02-06,family_5,class_3,type_3,product_24,1875a249-e60b-4254-92c7-95289cf4f9e9,165,0
4,family_1,class_2,type_1,429.0,2021-11-21,1,2022-10-25,family_4,class_2,type_1,product_16,22a34829-f78b-4551-aae4-3466a6543981,87,0
7,family_1,class_2,type_1,568.0,2022-08-19,1,2020-03-05,family_9,class_7,type_1,product_5,0221948b-97fb-4114-9cb5-7da78394b17f,141,0


In [ ]:
# Ne garder que une seule ligne de visite web par conversion (tips : Group by --> max)
df_result['store_conversion'] = df_result.groupby(['family_web', 'class_web', 'type_web', 'user_id', 'web_visit_date'])['store_conversion'].transform('max')
df_result.drop_duplicates()


,family_web,class_web,type_web,user_id,web_visit_date,visit_count_30d,date,family_store,class_store,type_store,product,product_id,price,store_conversion
0,family_1,class_2,type_1,212.0,2021-08-29,1,2021-06-24,family_9,class_8,type_2,product_9,26cd5030-e547-4725-8894-031f352330af,1,0
1,family_1,class_2,type_1,334.0,2021-08-07,1,2020-12-14,family_1,class_3,type_1,product_1,19f31143-41d6-4e4a-aedf-1eca0d014971,13,0
2,family_1,class_2,type_1,366.0,2021-01-27,1,2022-02-06,family_5,class_3,type_3,product_24,1875a249-e60b-4254-92c7-95289cf4f9e9,165,0
4,family_1,class_2,type_1,429.0,2021-11-21,1,2022-10-25,family_4,class_2,type_1,product_16,22a34829-f78b-4551-aae4-3466a6543981,87,0
7,family_1,class_2,type_1,568.0,2022-08-19,1,2020-03-05,family_9,class_7,type_1,product_5,0221948b-97fb-4114-9cb5-7da78394b17f,141,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148262,family_9,class_9,type_1,35636.0,2021-01-29,1,2022-05-23,family_2,class_1,type_1,product_13,2c0ac231-2482-4b4a-94f5-4da13ad4ff85,8,0
148277,family_9,class_9,type_1,35636.0,2021-09-13,1,2022-05-23,family_2,class_1,type_1,product_13,2c0ac231-2482-4b4a-94f5-4da13ad4ff85,8,0
148292,family_9,class_9,type_1,35636.0,2022-09-28,1,2022-05-23,family_2,class_1,type_1,product_13,2c0ac231-2482-4b4a-94f5-4da13ad4ff85,8,0
148307,family_9,class_9,type_1,38784.0,2020-06-21,1,2021-08-17,family_9,class_9,type_1,product_10,2e7a1337-4075-41af-83f2-2727f971d1da,126,0


In [ ]:
# Résultat final
df_card_owner_features = df_result.drop_duplicates()
df_card_owner_features
print("----Si il y a un résultat alors aller à la section 'Vérification de notre base de modélisation'-----")

,family_web,class_web,type_web,user_id,web_visit_date,visit_count_30d,date,family_store,class_store,type_store,product,product_id,price,store_conversion
0,family_1,class_2,type_1,212.0,2021-08-29,1,2021-06-24,family_9,class_8,type_2,product_9,26cd5030-e547-4725-8894-031f352330af,1,0
1,family_1,class_2,type_1,334.0,2021-08-07,1,2020-12-14,family_1,class_3,type_1,product_1,19f31143-41d6-4e4a-aedf-1eca0d014971,13,0
2,family_1,class_2,type_1,366.0,2021-01-27,1,2022-02-06,family_5,class_3,type_3,product_24,1875a249-e60b-4254-92c7-95289cf4f9e9,165,0
4,family_1,class_2,type_1,429.0,2021-11-21,1,2022-10-25,family_4,class_2,type_1,product_16,22a34829-f78b-4551-aae4-3466a6543981,87,0
7,family_1,class_2,type_1,568.0,2022-08-19,1,2020-03-05,family_9,class_7,type_1,product_5,0221948b-97fb-4114-9cb5-7da78394b17f,141,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148262,family_9,class_9,type_1,35636.0,2021-01-29,1,2022-05-23,family_2,class_1,type_1,product_13,2c0ac231-2482-4b4a-94f5-4da13ad4ff85,8,0
148277,family_9,class_9,type_1,35636.0,2021-09-13,1,2022-05-23,family_2,class_1,type_1,product_13,2c0ac231-2482-4b4a-94f5-4da13ad4ff85,8,0
148292,family_9,class_9,type_1,35636.0,2022-09-28,1,2022-05-23,family_2,class_1,type_1,product_13,2c0ac231-2482-4b4a-94f5-4da13ad4ff85,8,0
148307,family_9,class_9,type_1,38784.0,2020-06-21,1,2021-08-17,family_9,class_9,type_1,product_10,2e7a1337-4075-41af-83f2-2727f971d1da,126,0


### Méthode 2 (uniquement si la 1 ne fonctionne pas)

In [ ]:
# Methode 2

credentials = service_account.Credentials.from_service_account_file(
    "sa-creds-c-utt-dataset.json"
)

# Initialise le client BigQuery
client = bigquery.Client(credentials=credentials, project="c-hec-dataset")

# Définir l'identifiant de la table à charger
table_id = "c-hec-dataset.LM_use_case.card_owners_features"

limit_rows = 100000 # A modifier pour ajuster le nombre de ligne à charger dans la dataframe
query = query = f"SELECT * FROM `{table_id}` LIMIT {limit_rows}"

df_card_owner_features = client.query(query).to_dataframe()

df_card_owner_features


In [ ]:
# Ajoute les colonnes dérivées
df_card_owner_features['class_concat'] = df_card_owner_features['family'] + df_card_owner_features['class']
df_card_owner_features['type_concat'] = df_card_owner_features['family'] + df_card_owner_features['class'] + df_card_owner_features['type']
df_card_owner_features['month'] = pd.to_datetime(df_card_owner_features['web_visit_date']).dt.month
df_card_owner_features['year'] = pd.to_datetime(df_card_owner_features['web_visit_date']).dt.year

# Créé une liste unique de "type_concat"
df_unique_types = df_card_owner_features[['type_concat']].drop_duplicates().reset_index(drop=True)

# Ajoute les indices de type
df_unique_types['type_index'] = range(1, len(df_unique_types) + 1)

# Joins les indices à la table de base
df_final = df_card_owner_features.merge(df_unique_types, on='type_concat', how='left')

df_final

### Vérification de notre base de modélisation

In [ ]:
# Data exploration de la base finale (Partie 1)

# 1. Nombre total de conversions
total_conversions = df_card_owner_features['store_conversion'].sum()
print(f"Nombre total de conversions : {total_conversions}")
print("-------------------------")


Nombre total de conversions : 15967
-------------------------


In [ ]:
# 2. Nombre d'utilisateurs logués uniques
unique_users_logged_in = df_card_owner_features['user_id'].nunique()
print(f"Nombre d'utilisateurs logués uniques : {unique_users_logged_in}")
print("-------------------------")


Nombre d'utilisateurs logués uniques : 5117
-------------------------


In [ ]:
# 3. Top familles/classes/types les plus convertis
top_converted_families = df_card_owner_features[df_card_owner_features['store_conversion'] == 1] \
    .groupby(['family_web', 'class_web', 'type_web'])  \
    .size()  \
    .reset_index(name='nb_conversions') \
    .sort_values(by='nb_conversions', ascending=False)
print("Familles/Classes/Types les plus convertis :")
print(top_converted_families.head())


Familles/Classes/Types les plus convertis :
   family_web class_web type_web  nb_conversions
53   family_9   class_1   type_3              27
46   family_6   class_2   type_4              24
2    family_1   class_2   type_3              24
30   family_4   class_8   type_2              24
16   family_4   class_1   type_1              22


In [ ]:
# Data exploration de la base finale (Partie 2)

# 5. Taux de conversion global
total_visits = df_card_owner_features['user_id'].count()
conversion_rate = (top_converted_families['nb_conversions'].sum() / total_visits) * 100
print(f"Taux de conversion global : {conversion_rate:.2f}%")
print("-------------------------")

Taux de conversion global : 4.58%
-------------------------


In [ ]:
# 6. Répartition des conversions par mois
df_card_owner_features['month'] = df_card_owner_features['web_visit_date'].dt.month

conversions_by_month = df_card_owner_features[df_card_owner_features['store_conversion'] == 1] \
    .groupby('month') \
    .size() \
    .reset_index(name='nb_conversions') \
    .sort_values(by='month')
print("Répartition des conversions par mois :")
print(conversions_by_month)
print("-------------------------")

Répartition des conversions par mois :
    month  nb_conversions
0       1              53
1       2              41
2       3              43
3       4              51
4       5              62
5       6              70
6       7              73
7       8              74
8       9              51
9      10              75
10     11              73
11     12              65
-------------------------


In [ ]:
# 7. Répartition des utilisateurs logués par mois
users_by_month = df_card_owner_features.groupby('month')['user_id'].nunique().reset_index(name='nb_users')
print("Répartition des utilisateurs logués par mois :")
print(users_by_month)

Répartition des utilisateurs logués par mois :
    month  nb_users
0       1       906
1       2       816
2       3       839
3       4       946
4       5      1070
5       6      1055
6       7      1000
7       8       836
8       9       782
9      10       904
10     11      1034
11     12      1076


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

## Préparation du modèle

In [ ]:
# On selectionne les variables explicatives
x_cols = ['family_web',
          'class_web',
          'type_web',
          'month',
          'visit_count_30d']
# On selectionne la variable expliquée/à expliquer
y_col = ['store_conversion']

In [ ]:
# Création de 2 dataframes de "work" et "validate" par rapport à l'année 2022
df_card_owner_features['year'] = df_card_owner_features['web_visit_date'].dt.year
df_work = df_card_owner_features.loc[df_card_owner_features['year'] == 2022]
print(f"df_work : {df_work.shape}")
df_validate = df_card_owner_features.loc[df_card_owner_features['year'] == 2023]
print(f"df_validate : {df_validate.shape}")

df_work : (5325, 16)
df_validate : (252, 16)


In [ ]:
# Division du dataframe work en train et test (80/20)
df_train, df_test = train_test_split(df_work, train_size=0.8, random_state=42)
print(f"df_train : {df_train.shape}")
print(f"df_test : {df_test.shape}")

df_train : (4260, 16)
df_test : (1065, 16)


In [ ]:
# Selection des colonnes pour chaque dataset
x_test = df_test[x_cols]
x_train = df_train[x_cols]
x_validate = df_validate[x_cols]

y_test = df_test[y_col]
y_train = df_train[y_col]
y_validate = df_validate[y_col]

KeyError: "['type_index', 'nb_visits_type_30d'] not in index"

### Entrainement du modèle

#### Ne pas executer

In [ ]:
# Choix et entrainement du modèle
.... # import du bon modèle choisi
model = ... # initialisation du modèle
result = ... # entrainement du modèle sur le data set d'entrainement

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:family_web: object, class_web: object, type_web: object

#### A executer

In [ ]:
# Chargement du modèle
with open('model.pickle', 'rb') as f:
    model = pickle.load(f)

# Entrainement du modèle sur la dataframe train
result = ... # entrainement du modèle sur le data set d'entrainement

### Test du modèle

In [ ]:
# Effectue des prédictions sur l'échantillon test
y_pred = model.predict(x_test)
predictions = [round(value) for value in y_pred]
print(f"Nombre de conversion prédit la base test (base train) : {sum(predictions)}")

ValueError: feature_names mismatch: ['type_index', 'month', 'year', 'nb_visits_type_30d'] ['family_web', 'class_web', 'type_web', 'month', 'visit_count_30d']
expected nb_visits_type_30d, year, type_index in input data
training data did not have the following fields: type_web, class_web, family_web, visit_count_30d

In [ ]:
# Déterminer la précision du modèle
accuracy = ...
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# Afficher la matrice de confusion
confusion = ...
total = y_test.shape[0]
print(confusion/total)

ax= plt.subplot()
sns.heatmap(confusion/total, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['conversion', 'no conversion'])
ax.yaxis.set_ticklabels(['conversion', 'no conversion'])

### Validation du modèle

In [ ]:
# Effectue des prédictions sur l'échantillon de validation
y_validate = ..
predictions_validation = [value for value in y_validate]
print(f"Nombre de conversion prédit la base test (base validate) : {sum(predictions_validation)}")

In [ ]:
# Déterminer la précision du modèle
accuracy = ...
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# Afficher la matrice de confusion
confusion = ...
total = y_validate.shape[0]
print(confusion/total)


ax= plt.subplot()
sns.heatmap(confusion/total, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['conversion', 'no conversion'])
ax.yaxis.set_ticklabels(['conversion', 'no conversion'])

In [ ]:
#Available importance_types = [‘weight’, ‘gain’, ‘cover’, ‘total_gain’, ‘total_cover’]
f = 'gain'
model.get_booster().get_score(importance_type= f)

## Pour aller plus loin

A vous de jouer :
Rappelez vous, jusqu'a maintenant nous avons testé et validé notre modèle uniquement sur la population de visiteur qui ont converti en magasin.
Maintenant il faut executer notre modèle sur l'ensemble de notre base visiteur afin de calculer au mieux notre retour sur investissement média.